In [ ]:
import boto3
ec2 = boto3.resource('ec2')

In [ ]:
# We need a security group for the instance that we are going to
# create the AMI from.  We do this to facilitate waiting for the
# user data configuration script to finish executing.  We will
# assume that the security group doesn't already exist.
security_group_name = 'java-app-security-group'
security_group = ec2.create_security_group(
    GroupName=security_group_name,
    Description='Security group for java application instances'
)
response = security_group.authorize_ingress(
    IpPermissions=[
        {'FromPort': 4567,
        'ToPort': 4567,
        'IpProtocol': 'tcp',
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

In [ ]:
# This is the user data script for configuring the instance.
user_data  = """#! /bin/bash
# upgrade machine
sudo yum update -y

# install java 8 jdk
sudo yum install -y java-1.8.0-openjdk-devel

# Download app
cd /home/ec2-user
wget https://github.com/simplesteph/ec2-masterclass-sampleapp/releases/download/v1.0/ec2-masterclass-sample-app.jar

# System D type of Configuration for Linux 2
sudo bash -c 'cat << \EOF > /etc/systemd/system/ec2sampleapp.service
[Unit]
Description=EC2 Sample App
After=network.target

[Service]
ExecStart=/usr/bin/java -Xmx700m -jar /home/ec2-user/ec2-masterclass-sample-app.jar
Restart=on-failure

[Install]
WantedBy=multi-user.target
EOF'

# apply across reboots
sudo systemctl enable ec2sampleapp.service # enable on boot
sudo systemctl start ec2sampleapp.service
"""

In [ ]:
# Create an instance using user data
instances = ec2.create_instances(
    ImageId='ami-00dfe2c7ce89a450b',
    InstanceType='t2.micro',
    MaxCount=1,
    MinCount=1,
    UserData=user_data,
    SecurityGroups=[
        security_group_name
    ]
)
instance = instances[0]
instance.wait_until_running()
instance.reload()

In [ ]:
# Wait until the user data script completes.  We assume exceptions
# occur strictly because the script has not completed.
import urllib, time
instance_url = 'http://' + instance.public_ip_address + ':4567'
while True:
    time.sleep(5)
    try:
        urllib.request.urlopen(instance_url)
    except:
        continue
    else:
        break

In [ ]:
# Stop the instance
instance.stop()
instance.wait_until_stopped()

In [ ]:
# Create a snapshot of the root volume of the instance.
block_device_mapping = instance.block_device_mappings[0]
snapshot = ec2.create_snapshot(
    VolumeId=block_device_mapping['Ebs']['VolumeId']
)
snapshot.wait_until_completed()

In [ ]:
# Register the image
device_name = block_device_mapping['DeviceName']
image = ec2.register_image(
    BlockDeviceMappings=[
        {
            'DeviceName': device_name,
            'Ebs': {
                'DeleteOnTermination': True,
                'SnapshotId': snapshot.snapshot_id
            }
        }
    ],
    Description='Java Application AMI',
    Name='java-application-ami',
    RootDeviceName=device_name
)
image.wait_until_exists()

In [ ]:
# Terminate the instance
instance.terminate()
instance.wait_until_terminated()

In [ ]:
# The product of this notebook is the AMI and not the security
# group.  Because other notebooks will assume the security
# group doesn't exist, delete it.
response = security_group.delete()

In [ ]:
# Success!
print('AMI created')